In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import pickle


In [4]:
## Read the dataset, convert it into dataframe
dataframe = pd.read_csv(r'C:\Users\sid\Desktop\ass1\Fake_News_Detection-master\news.csv')
dataframe.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
## Split data into X & Y
x = dataframe['text']
y = dataframe['label']

In [6]:
#data pre processing by checking the null values
dataframe.isnull().any()

Unnamed: 0    False
title         False
text          False
label         False
dtype: bool

In [7]:
print(x)

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
6335                      Mumbai indians lost all matches
Name: text, Length: 6336, dtype: object


In [8]:
print(y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
6335    REAL
Name: label, Length: 6336, dtype: object


In [9]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [10]:
## Now we need to fit the TFIDF Vectorizer.
# max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
# max_df = 25 means "ignore terms that appear in more than 25 documents".

tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [11]:
## Now let's fit the Machine Learning Model
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [12]:
# Now let's check model accuracy. Let's fit model on the test data.

y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.93%


In [13]:
pipeline = Pipeline ([('tfidf', TfidfVectorizer (stop_words='english' )),
                     ('nbmodel', MultinomialNB())])

In [14]:
pipeline.fit(x_train, y_train)
score=pipeline.score(x_test,y_test)
print('accuracy',score)

accuracy 0.8485804416403786


In [15]:
pred = pipeline.predict(x_test)
print(classification_report (y_test, pred))

              precision    recall  f1-score   support

        FAKE       0.96      0.72      0.82       618
        REAL       0.78      0.97      0.87       650

    accuracy                           0.85      1268
   macro avg       0.87      0.85      0.85      1268
weighted avg       0.87      0.85      0.85      1268



In [16]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[584  34]
 [ 43 607]]


In [17]:
with open(r'C:\Users\sid\Desktop\ass1\Fake_News_Detection-master\model.pkl', 'wb') as handle:
          pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)          

In [18]:
## Let's create function for test the model on the real-time data.

def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [19]:
fake_news_det("Mumbai indians lost all matches")

['FAKE']
